In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

**Load the trainset**

In [12]:
train = pd.read_csv('/home/gautam/Documents/ML-Projects/Titanic Passenger Survival/Datasets/train.csv')
test = pd.read_csv('/home/gautam/Documents/ML-Projects/Titanic Passenger Survival/Datasets/test.csv')
target = pd.read_csv('/home/gautam/Documents/ML-Projects/Titanic Passenger Survival/Datasets/gender_submission.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


**Data Preprocessing**

In [14]:
train = train.dropna(subset=['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Age', 'Fare'])
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
x_train = train.drop(["Survived"], axis=1)
y = train["Survived"]
y.head(891)

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 712, dtype: int64

In [16]:
x_test = test.dropna(subset=['Pclass', 'SibSp', 'Parch', 'Age', 'Fare'])
y_test = x_test["Survived"]
y_test.head(418)

0      0
1      1
2      0
3      0
4      1
      ..
409    1
411    1
412    1
414    1
415    0
Name: Survived, Length: 331, dtype: int64

In [17]:
train_features = ['Pclass', 'SibSp', 'Parch', 'Age', 'Fare']
X = x_train[train_features]
X.describe()

,Pclass,SibSp,Parch,Age,Fare
count,712.000000,712.000000,712.000000,712.000000,712.000000
mean,2.240169,0.514045,0.432584,29.642093,34.567251
std,0.836854,0.930692,0.854181,14.492933,52.938648
min,1.000000,0.000000,0.000000,0.420000,0.000000
25%,1.000000,0.000000,0.000000,20.000000,8.050000
50%,2.000000,0.000000,0.000000,28.000000,15.645850
75%,3.000000,1.000000,1.000000,38.000000,33.000000
max,3.000000,5.000000,6.000000,80.000000,512.329200


**Training a randow forest classifier on the training train**

In [18]:
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
clf.fit(X, y)

RandomForestClassifier(max_depth=5, random_state=1)

In [19]:
y_pred = clf.predict(x_test[train_features])

In [20]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc*100))

Accuracy: 63.75%
